In [ ]:
!pip install cantera

import matplotlib.pyplot as plt
import cantera as ct
import numpy as np

In [ ]:
# Define gas properties
T = 300.0 
p = ct.one_atm 
phi = 2.0
X_fuel = {"H2": 1.0} 
X_ox = {"O2": 1.0, "N2": 3.76}
mechanism = "h2o2.yaml"
width = 0.03 # m

In [ ]:
transport_models = ["mixture-averaged", "unity-Lewis-number", "multicomponent"]
flames = {}
properties = {}

for transport_model in transport_models:
    gas = ct.Solution(mechanism)
    gas.TP = T, p
    gas.set_equivalence_ratio(phi, X_fuel, X_ox)

    flame = ct.FreeFlame(gas, width=width)
    flame.transport_model = transport_model

    flame.energy_enabled = True
    flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

    flame.set_max_jac_age(10, 10)
    flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
    flame.max_time_step_count = 5000    

    # Set steady-state tolerance to relative and absolute tolerances of 1e-15
    flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
    flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])


    flame.solve(loglevel=1)
    flames[transport_model] = flame

In [ ]:
linestyles = {
    "mixture-averaged": "-",
    "unity-Lewis-number": "--",
    "multicomponent": ":"
}

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(3, 4), sharex=True)

for transport_model, flame in flames.items():
    ax = axs[0]
    ax.plot(flame.grid, flame.T, label=transport_model, linestyle=linestyles[transport_model])

    ax = axs[1]
    ax.plot(flame.grid, flame.velocity, linestyle=linestyles[transport_model])

axs[0].set(ylabel=r"$T$ $[K]$")
axs[1].set(xlabel=r"$x$ $[m]$", ylabel=r"$u$ $[m/s]$")

axs[0].grid()
axs[1].grid()

# Fig.legend on top
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=3)
fig.subplots_adjust(top=0.85)
axs[0].set_title(r"$\phi = {}$" .format(phi))

plt.show()

# Part 2: Mixture fraction

$$
(1) \qquad b=2 \frac{Z_{\text {mass }, C}}{W_C}+0.5 \frac{Z_{\text {mass }, H}}{W_H}-\frac{Z_{\text {mass }, O}}{W_O}
$$

$$
(2) \qquad Z(\phi) = \frac{b(\phi) - b_{oxidizer}}{b_{fuel} - b_{oxidizer}}
$$

$$
(3) \qquad \phi=\frac{Z}{1-Z} \frac{1-Z_{\text {st }}}{Z_{\text {st }}}
$$

In [ ]:
# Coefficients of equation (1) 
bilger_weights = {"C": 2.0, "H": 0.5, "O": -1}

# Function (1) - b from Elemental mass fractions with gas object
def bilger_from_gas(gas, bilger_weights):

    b = 0.0

    for k, v in bilger_weights.items():
        if k in gas.element_names:
            Z_k = gas.elemental_mass_fraction(k)
            W_k = gas.atomic_weight(k)
        
            b += v * Z_k / W_k

    return b

# Function (1) - b from Elemental mass fractions with flame object
def bilger_from_flame(flame, gas, bilger_weights):

    b = np.zeros_like(flame.grid)

    for k, v in bilger_weights.items():
        if k in gas.element_names:
            Z_k = flame.elemental_mass_fraction(k)
            W_k = gas.atomic_weight(k)
        
            b += v * Z_k / W_k

    return b

# Function (3) - phi from mixture fraction
def phi_from_Z(Z, Z_stoichiometric):
    
    return Z * (1/Z_stoichiometric - 1) / (1 - Z)

In [ ]:
# Calculate mixture fraction values at pure fuel and pure oxidizer conditions,
# and at stoichiometric conditions
gas_fuel = ct.Solution(mechanism)
gas_fuel.TPX = T, p, X_fuel
gas_ox = ct.Solution(mechanism)
gas_ox.TPX = T, p, X_ox
gas_st = ct.Solution(mechanism)
gas_st.TP = T, p
gas_st.set_equivalence_ratio(1.0, fuel=X_fuel, oxidizer=X_ox)

b_fuel = bilger_from_gas(gas_fuel, bilger_weights)
b_ox = bilger_from_gas(gas_ox, bilger_weights)
b_st = bilger_from_gas(gas_st, bilger_weights)
Z_st = (b_st - b_ox) / (b_fuel - b_ox)

In [ ]:
#
# Solve flames but save data for mixture fraction analysis
#
transport_models = ["mixture-averaged", "unity-Lewis-number", "multicomponent"]
flames = {}
properties = {}

for transport_model in transport_models:
    gas = ct.Solution(mechanism)
    gas.TP = T, p
    gas.set_equivalence_ratio(phi, X_fuel, X_ox)

    flame = ct.FreeFlame(gas, width=width)
    flame.transport_model = transport_model

    flame.energy_enabled = True
    flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

    flame.set_max_jac_age(10, 10)
    flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
    flame.max_time_step_count = 5000    

    # Set steady-state tolerance to relative and absolute tolerances of 1e-15
    flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
    flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])


    flame.solve(loglevel=1)
    flames[transport_model] = flame

    b = bilger_from_flame(flame, gas, bilger_weights)
    Z = (b - b_ox) / (b_fuel - b_ox)
    phi_flame = phi_from_Z(Z, Z_st)

    properties[transport_model] = {}
    properties[transport_model]["Z"] = Z
    properties[transport_model]["phi"] = phi_flame

In [ ]:
#
# Plot mixture fraction and enthalpy
#
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(3, 4), sharex=True)

for transport_model, flame in flames.items():
    ax = axs[0]
    ax.plot(flame.grid, properties[transport_model]["Z"], label=transport_model, linestyle=linestyles[transport_model])

    ax = axs[1]
    ax.plot(flame.grid, flame.enthalpy_mass, linestyle=linestyles[transport_model])

axs[0].set(ylabel=r"$Z$ $[-]$")
axs[1].set(xlabel=r"$x$ $[m]$", ylabel=r"$h$ $[J/kg]$")
axs[0].grid()
axs[1].grid()

# Fig.legend on top
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=3)
fig.subplots_adjust(top=0.85)
axs[0].set_title(r"$\phi = {}$" .format(phi))

plt.show()

In [ ]:
# Plot all species mass fractions

nrows=3
ncols=4

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6), sharex=True)

for transport_model, flame in flames.items():
    for i in range(len(gas.species_names)):
        ax = axs[i//ncols][i%ncols]
        label = transport_model if i == 0 else None
        ax.plot(flame.grid, flame.Y[i, :], label=label, linestyle=linestyles[transport_model])

        ax.set_title(gas.species_names[i])
    
# Clear unused axes
for i in range(len(gas.species_names), nrows*ncols):
    axs[i//ncols][i%ncols].axis("off")

fig.subplots_adjust(hspace=0.2, top=0.91, wspace=0.6)
handles, labels = axs[0][0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=3)

In [ ]:
#
# Final plot with density and mean molecular weight
#
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(3, 4), sharex=True)

for transport_model, flame in flames.items():
    ax = axs[0]
    ax.plot(flame.grid, flame.density, label=transport_model, linestyle=linestyles[transport_model])

    ax = axs[1]
    ax.plot(flame.grid, flame.mean_molecular_weight, linestyle=linestyles[transport_model])

axs[0].set(ylabel=r"$\rho$ $[kg/m^3]$")
axs[1].set(xlabel=r"$x$ $[m]$", ylabel=r"$\W$ $[kg/kmol]$")
axs[0].grid()
axs[1].grid()

# Fig.legend on top
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncol=3)
fig.subplots_adjust(top=0.85)
axs[0].set_title(r"$\phi = {}$" .format(phi))

plt.show()